In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.4.1/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_SubsetB_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_SubsetB_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time, Trop_Peak_Time = [], [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Trop_Peak_Time.append(Datetime(ET_DF["Trop Peak Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Trop Peak Time"] = Trop_Peak_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
# Create Phase Distribution Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)
Lat_Bins = Create_Bins(10,60,5)
Lon_Bins = Create_Bins(-100,20,5)

In [7]:
Num_Years = numpy.array([90,93,93])

In [8]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [9]:
Control_ET

,Code,Name,ABC,Trans Type,ET Begin Time,ET Complete Time,Trop Peak Time,Peak Time,Peak SLP,Peak Lon,Peak Lat,Trop Peak SLP,Trop Peak Lon,Trop Peak Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat
0,TC190001,Alex,A,3,1900-07-03 18:00:00,1900-07-03 18:00:00,1900-06-25 06:00:00,1900-06-25 06:00:00,987.41,-84.91,29.95,987.41,-84.91,29.95,1015.57,-90.75,36.25,1015.57,-90.75,36.25
1,TC190003,Colin,A,1,1900-09-18 00:00:00,1900-09-20 12:00:00,1900-09-19 12:00:00,1900-09-19 12:00:00,934.83,-70.67,39.42,943.73,-77.15,24.55,952.06,-77.17,31.14,979.68,-62.68,47.61
2,TC190004,Dawn,A,1,1900-11-05 12:00:00,1900-11-06 12:00:00,1900-11-04 00:00:00,1900-11-04 00:00:00,948.88,-91.58,20.70,948.88,-91.58,20.70,954.74,-87.08,27.00,992.74,-74.44,30.88
3,TC190005,Ernest,A,1,1900-11-06 00:00:00,1900-11-08 00:00:00,1900-11-06 18:00:00,1900-11-06 18:00:00,988.17,-20.93,46.12,994.44,-32.46,35.66,996.02,-27.73,41.42,998.22,-9.00,45.00
4,TC190102,Harrison,A,1,1901-05-28 12:00:00,1901-05-30 18:00:00,1901-05-27 06:00:00,1901-05-27 06:00:00,980.89,-51.83,25.92,980.89,-51.83,25.92,991.56,-43.71,30.04,1008.78,-34.75,24.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,TC198802,Micah,C,1,1988-02-06 00:00:00,1988-02-06 06:00:00,1988-02-05 06:00:00,1988-02-05 06:00:00,998.09,-35.19,36.46,998.09,-35.19,36.46,999.63,-31.01,38.29,999.02,-28.74,38.72
278,TC198805,Seldoen,C,1,1988-09-07 06:00:00,1988-09-08 12:00:00,1988-09-07 00:00:00,1988-09-07 00:00:00,980.43,-49.24,32.21,980.43,-49.24,32.21,984.68,-48.37,33.35,987.91,-35.17,42.45
279,TC198808,Alice,C,2,1988-09-12 06:00:00,1988-09-12 12:00:00,1988-09-13 06:00:00,1988-09-13 06:00:00,990.70,-21.25,51.00,998.54,-45.95,35.61,1009.08,-39.50,44.00,1008.35,-37.50,45.75
280,TC198809,Bruce,C,1,1988-09-21 06:00:00,1988-09-27 00:00:00,1988-09-22 00:00:00,1988-09-22 00:00:00,934.40,-63.31,32.34,961.85,-68.34,28.60,961.07,-66.39,30.40,1001.36,-41.00,47.00


In [ ]:
# Storm Peak SLP, ET Begin SLP, ET Complete SLP
# Tropical Duration, Storm Duration
# Mean Storm PACE, Annual PACE
# Precip Rate